In [1]:
import pandas as pd
from datetime import datetime, timedelta

%run /Users/avneeshchandra/Cuttlefish/notebooks/users/avneesh/connect_to_api.ipynb

In [6]:
def get_retweet_data(feature = 'tweets'):
    query = "SELECT * FROM \
    \
    ( \
    SELECT \
    * \
    FROM \
    hits_twitter_{} \
    JOIN \
    map_nodes on map_nodes.node_id = hits_twitter_{}.node_id \
    ) s \
    \
    WHERE \
    s.hit_time BETWEEN '{}'::TIMESTAMP AND '{}'::TIMESTAMP;;".format(feature, feature, datetime.now() - timedelta(hours=1), datetime.now())
    
    r = cur.execute(query)
    hits = cur.fetchall()
    hits_df = pd.DataFrame(hits, columns=["id","hit_value", "node_id", "message_id","hit_time","params","id_2","node_id2","map_id","cluster_id"])

    hits_df["node_id"] = hits_df["node_id"].astype("str")
    hits_df["message_id"] = hits_df["message_id"].astype("str")
    hits_df["hit_type"] = feature

    return hits_df.set_index(['id'])

In [3]:
temp = get_retweet_data()

In [27]:
# temp.drop_duplicates('node_id',inplace = True)
temp

,hit_value,node_id,message_id,hit_time,params,id_2,node_id2,map_id,cluster_id,hit_type
id,,,,,,,,,,
10256297318,1228296551652569091,443352535,1228296551652569091,2020-02-14 12:34:53,None,4381837,443352535,1762,1,tweets
10256304682,1228296609907249157,1082247670155042816,1228296609907249157,2020-02-14 12:35:07,None,5389314,1082247670155042816,1939,38,tweets
10256304801,1228296584309485569,1108397881994694658,1228296584309485569,2020-02-14 12:35:01,None,5386567,1108397881994694658,1939,11,tweets
10256304873,1228296589380329473,562202944,1228296589380329473,2020-02-14 12:35:02,None,440361,562202944,840,21,tweets
10256668721,1228297398465781760,197823320,1228297398465781760,2020-02-14 12:38:15,None,7011565,197823320,2313,15,tweets
...,...,...,...,...,...,...,...,...,...,...
10262431342,1228311551796363266,1216651544,1228311551796363266,2020-02-14 13:34:29,None,7073200,1216651544,2331,15,tweets
10262431801,1228311326444982272,2776286888,1228311326444982272,2020-02-14 13:33:35,None,5459237,2776286888,1907,44,tweets
10262440458,1228311332354699264,8859092,1228311332354699264,2020-02-14 13:33:37,None,7053872,8859092,2325,54,tweets


In [36]:
# map_cluster_pair_list = temp[['map_id','cluster_id']].values.tolist()
map_cluster_dict = {}
for map_id in set(temp['map_id'].values.tolist()):
    map_cluster_dict[map_id] = (set(temp[temp.map_id == map_id].cluster_id.values.tolist()))

In [46]:
map_clustercount_dict = {}

for k,v in map_cluster_dict.items():
    map_clustercount_dict[k] = len(v)


In [58]:
top_maps_by_clusters = pd.DataFrame.from_dict(map_clustercount_dict,orient = 'index',columns = ['count']).sort_values(['count'],ascending = False)

In [59]:
print('Most active 5 maps are ',top_maps_by_clusters.index.tolist()[:5])

Most active maps are  [1930, 2188, 2331, 2313]
